In [ ]:
from qiskit import QuantumCircuit, transpile
import numpy as np

num_qubits = 8
qc = QuantumCircuit(num_qubits)
num_layers = int(np.ceil(np.log2(num_qubits)))


In [ ]:
from qcc.ml import USE_CUDA

USE_CUDA


In [ ]:
from itertools import zip_longest, tee

def convolve(wires):
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    
    qc.cnot(wires[0], wires[1])
    qc.ry(0.1, wires[0])
    qc.rz(0.1, wires[1])
    qc.cnot(wires[1], wires[0])
    qc.ry(0.1, wires[0])
    qc.cnot(wires[0], wires[1])
    
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
        
def pool(wires):
    qc.crz(0.1, wires[0], wires[1])
    qc.x(wires[0])
    qc.crx(0.1, wires[0], wires[1])

def _convolution(iterable):
    a, b = tee(iterable)
    first = next(b, None)
    lst = list(zip_longest(a, b, fillvalue=first))
    last = lst.pop()[::-1]
    lst = lst[::2] + lst[1::2]

    if len(lst) > 1:
        lst.insert(0, last)

    for wires in lst:
        convolve(wires)

def _pooling(iterable):
    measurements = iterable[1::2]
    controlled = iterable[0::2]

    for wires in zip(measurements, controlled):
        pool(wires)

    return controlled

wires = qc.qubits
for _ in range(num_layers):
    _convolution(wires)
    wires = _pooling(wires)


In [ ]:
qc.draw('mpl')


In [ ]:
qc = transpile(qc, basis_gates=["cx", "sx", "x", "ry", "rx", "rz"], optimization_level=0)
qc.depth(), sum(qc.count_ops().values())


In [ ]:
135, 415
188, 593


In [ ]:
qc = QuantumCircuit(num_qubits)

def _ry_pool(iterable):
    measurements = iterable[1::2]
    controlled = iterable[0::2]

    for wires in zip(measurements, controlled):
        qc.cry(0.1, wires[0], wires[1])

    return controlled

wires = qc.qubits
for _ in range(num_layers):
    wires = _ry_pool(wires)
    
qc = transpile(qc, basis_gates=["cx", "sx", "x", "ry", "rx", "rz"], optimization_level=0)
qc.depth(), sum(qc.count_ops().values())


In [ ]:
from pathlib import Path

path = Path("/Users/mingyoungjeng/Documents/Spring 2023.nosync/quantum-convolutional-classifier/test/tmp")
for file in path.glob("**/mqcc*/*.*"):
    stem = file.stem.replace("mqcc_", "mqcc-optimized_")
    # items = file.stem.split("_")
    # items.insert(2, "16x16")
    # stem = "_".join(items)
    
    # items = file.parent.stem.split("_")
    # items.insert(2, "16x16")
    # parent = "_".join(items)
    
    new_path = file.parent.with_stem(file.parent.stem.replace("mqcc", "mqcc_optimized"))
    new_path.mkdir(parents=True, exist_ok=True)
    
    new_path = new_path / f"{stem}{file.suffix}"
    file.rename(new_path)


In [ ]:
from pathlib import Path

path = Path("/Users/mingyoungjeng/Downloads/results_thesis/_raw_results/CIFAR10")
dest = path / "32x32"

for file in path.glob("**/*_32x32_*.*"):
    parent = dest / file.parent.stem
    parent.mkdir(parents=True, exist_ok=True)
    
    new_path = parent / file.name
    file.rename(new_path)


In [ ]:
import numpy as np

test = np.array([[[0, 1], [2, 3]], [[4, 5], [6, 7]]])
print(test)

test.reshape(4, 2)


In [ ]:
from pathlib import Path
from PIL import Image

filename = Path("../img/color/Jayhawk_512x512x3.jpg")

with Image.open(filename, "r") as im:
    im.crop((0  , 0  , 256, 256)).save("tl.png")
    im.crop((128, 0  , 384, 256)).save("tc.png")
    im.crop((256, 0  , 512, 256)).save("tr.png")
    
    im.crop((0  , 128, 256, 384)).save("cl.png")
    im.crop((128, 128, 384, 384)).save("cc.png")
    im.crop((256, 128, 512, 384)).save("cr.png")
    
    im.crop((0  , 256, 256, 512)).save("bl.png")
    im.crop((128, 256, 384, 512)).save("bc.png")
    im.crop((256, 256, 511, 512)).save("br.png")


In [ ]:
from qiskit import QuantumCircuit


qc = QuantumCircuit(2)

qc.h(1)
qc.cx(1, 0)

qc.draw('mpl',reverse_bits=True)


In [1]:
from pathlib import Path
from qcc.cli.pooling import _pooling

inputs = Path("/Users/mingyoungjeng/Documents/Spring 2023.nosync/quantum-convolutional-classifier/img/color")

for n in [False]:
    for method in ["none"]:
            _pooling(method, [0], inputs=inputs, output_dir=Path("results"), noiseless=n)


none_0, 32x32x3, RGB: fidelity=98.312%
none_0, 512x512x3, RGB: fidelity=11.821%
none_0, 256x256x3, RGB: fidelity=37.025%
none_0, 16x16x3, RGB: fidelity=99.502%
none_0, 64x64x3, RGB: fidelity=93.560%
none_0, 128x128x3, RGB: fidelity=75.156%
none_0, 8x8x3, RGB: fidelity=99.858%


In [18]:
from pathlib import Path

path = Path("/Users/mingyoungjeng/Downloads/pooling_results/results_pooling_1000000")
for file in path.glob("**/*_noisy.png"):
    stem = file.stem.replace("_noisy", "_100000")

    # parent = dest / file.parent.stem
    # parent.mkdir(parents=True, exist_ok=True)
    
    # new_path = parent / file.name
    file.rename(file.with_stem(stem))


In [25]:
from pathlib import Path

root = Path("/Users/mingyoungjeng/Downloads/results_pooling")
path = Path("/Users/mingyoungjeng/Downloads/pooling_results")
for file in path.glob("**/*.png"):
    parent = file.parent.stem
    
    stem = file.stem.split("_")
    parent1, parent2, stem = stem[0], stem[-1], [parent, *stem[1:-1]]
    stem = "_".join(stem)
    
    parent = root / parent1 / parent2
    parent.mkdir(parents=True, exist_ok=True)
    
    new_path = parent / f"{stem}.png"
    # print(new_path)
    file.rename(new_path)


In [39]:
import polars as pl

x = "noiseless"
 
# read DataFrame
df = pl.read_csv(f"/Users/mingyoungjeng/Downloads/results_pooling/results_{x}.csv")


 
for data_size, group in df.group_by(['data_size']):
    group = group.select("dimension_reduction","fidelity")
    group.write_csv(f'{x}_{data_size[0]}.csv')
 
# print(pd.read_csv("Male.csv"))
# print(pd.read_csv("Female.csv"))


shape: (8, 2)
┌─────────────────────┬──────────┐
│ dimension_reduction ┆ fidelity │
│ ---                 ┆ ---      │
│ str                 ┆ f64      │
╞═════════════════════╪══════════╡
│ avg_1x1             ┆ 0.880519 │
│ avg_2x2             ┆ 0.778049 │
│ avg_3x3             ┆ 0.663056 │
│ avg_4x4             ┆ 0.61177  │
│ euclidean_1x1       ┆ 0.870863 │
│ euclidean_2x2       ┆ 0.770927 │
│ euclidean_3x3       ┆ 0.655336 │
│ euclidean_4x4       ┆ 0.611663 │
└─────────────────────┴──────────┘
shape: (6, 2)
┌─────────────────────┬──────────┐
│ dimension_reduction ┆ fidelity │
│ ---                 ┆ ---      │
│ str                 ┆ f64      │
╞═════════════════════╪══════════╡
│ avg_1x1             ┆ 0.921811 │
│ avg_2x2             ┆ 0.808869 │
│ avg_3x3             ┆ 0.757949 │
│ euclidean_1x1       ┆ 0.918786 │
│ euclidean_2x2       ┆ 0.802441 │
│ euclidean_3x3       ┆ 0.757931 │
└─────────────────────┴──────────┘
shape: (14, 2)
┌─────────────────────┬──────────┐
│ dimension_